<a href="https://colab.research.google.com/github/christiejibaraki/CUREBench/blob/main/notebooks/0-colab-environment-setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run CUREBench starter code with gpt oss model
- OpenAI GPT-OSS-20B open-weight reasoning model trained with Harmony response format
  - 20.9B params, 3.6B active per token
  - Mixture-of-Experts (MoE) transformer
  - `bitsandbytes` enables 8-bit quantization so GPT-OSS-20B can run on 16GB GPU

In [1]:
!nvidia-smi # driver version

Thu Oct 16 20:39:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# tooklkit version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
import os
os.environ['BNB_CUDA_VERSION'] = '125'

## Setup
- Clone forked CUREBench repo onto local `content` folder (this is not persistent)
- Create virtual environment and install packages
  - Package installation takes about **5min** ⏰
- Add virtual environment's site-packages to notebook's system path

In [4]:
!git clone https://github.com/christiejibaraki/CUREBench.git

Cloning into 'CUREBench'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 102 (delta 46), reused 28 (delta 16), pack-reused 16 (from 1)
Receiving objects: 100% (102/102), 84.91 KiB | 1.35 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [5]:
%cd CUREBench

/content/CUREBench


In [6]:
# !git pull

In [7]:
!python --version

Python 3.12.12


In [8]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 38.5 MB/s eta 0:00:00


In [9]:
!virtualenv env

created virtual environment CPython3.12.12.final.0-64 in 356ms
  creator CPython3Posix(dest=/content/CUREBench/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [10]:
!./env/bin/pip list

Package Version
------- -------
pip     25.2


In [ ]:
!./env/bin/pip install -r requirements.txt

In [12]:
import sys
env_path = './env/lib/python3.12/site-packages'
sys.path.append(env_path)

## GPT OSS Tutorial from CUREBench
- loading the model will take a few minutes

In [13]:
import os.path
from core.eval_framework import GPTOSS20BModel
from tooluniverse.execute_function import ToolUniverse
from openai_harmony import load_harmony_encoding

<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/hyperopt/atpe.py:19: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow

#### create directory for openai harmony encodings and PASTE encodings into the directory
There is an error with downloading direction from openai-harmony library

In [14]:
correct_cache_path = os.path.expanduser(os.environ.get('HOME', '/root')) + '/.cache/openai_harmony'
os.environ['TIKTOKEN_ENCODINGS_BASE'] = correct_cache_path # Re-set with resolved path for robustness
os.environ['TIKTOKEN_RS_CACHE_DIR'] = correct_cache_path

In [15]:
!echo $TIKTOKEN_ENCODINGS_BASE
!mkdir -p "{correct_cache_path}"

/root/.cache/openai_harmony


In [16]:
#make sure encodings are in directory
load_harmony_encoding('HarmonyGptOss')

In [17]:
model = GPTOSS20BModel("openai/gpt-oss-20b", reasoning_lvl="low", quantization="auto",
                       system_identity="You are a clinical trial assistant.")
model.load()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

_topk_forward.py: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [18]:
final, trace = model.inference("What’s 45 * 67?", builtin_tools=["python"])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [20]:
trace

[{'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text',
    'text': 'Need to compute 45*67. 45*60=2700; 45*7=315; sum=3015.'}],
  'channel': 'analysis'},
 {'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text', 'text': '45 × 67 = **3,015**'}],
  'channel': 'final'}]